In [1]:
import numpy as np

def compute_EOS(rhos, gamma, rho_t_start, P_t_start):
    """
    Calculate the pressure of a neutron star using a single polytropic equation of state (EOS).

    This function computes the pressure (`pres`) as a function of density (`rho`) using a single polytropic
    relation P = k * rho^gamma for rho > rho_t_start, where gamma is sampled uniformly from (1.77, 3.23).

    Parameters
    ----------
    rhos : array-like
        An array of density values (in cgs units) at which to calculate the pressure.
    
    theta : array-like, length 1
        A list or tuple containing a single polytropic index `gamma` (float) sampled uniformly 
        from the range (1.77, 3.23).
    
    rho_t_start : float, optional
        Density threshold for the polytropic EOS (in cgs units, default: 4.3721e11 g/cm^3).
    
    P_t_start : float, optional
        Pressure at the density threshold (in cgs units, default: 7.7582e29 dyn/cm^2).

    Returns
    -------
    pres : ndarray
        An array of pressure values (in cgs units) corresponding to the input density values.
    """
    # Extract the single gamma value
    #gamma = theta[0]
    
    # Calculate the polytropic constant k
    k = P_t_start / (rho_t_start ** gamma)
    
    # Compute pressure using the polytropic EOS: P = k * rho^gamma for rho > rho_t_start
    pres = np.zeros_like(rhos)
    cond = rhos > rho_t_start
    pres[cond] = k * (rhos[cond] ** gamma)
    
    return pres

In [3]:
import EOSgenerators.crust_EOS as crust
import EOSgenerators.RMF_EOS as RMF
import EOSgenerators.Polytrope_EOS as Polytrope
import EOSgenerators.Strangeon_EOS as Strangeon
import TOVsolver.main as main
import matplotlib.pyplot as plt
import numpy as np
import math
from TOVsolver.unit import g_cm_3,dyn_cm_2,km,Msun,MeV,fm
#from TOVsolver.maximum_central_density import maximum_central_density

In [5]:
data_k = np.loadtxt("data_k")
pressure = data_k[:,0]*dyn_cm_2
density = data_k[:,1]*g_cm_3

In [7]:
gamma=  np.random.uniform(1.77,3.23,1)

In [9]:
print(gamma)

[2.15015668]


In [11]:
eps_set = np.logspace(14.13,14.467,100,base=10)*g_cm_3
pres_out =compute_EOS(eps_set,gamma,density[-1], pressure[-1])

In [13]:
eps_total = np.hstack((density,eps_set))
pres_total = np.hstack((pressure, pres_out))

In [15]:
print(eps_total/g_cm_3)

[1.03170000e+04 1.04710000e+05 1.04570000e+06 ... 2.88530627e+14
 2.90801043e+14 2.93089325e+14]


In [17]:
print(pres_total/dyn_cm_2)

[9.53930000e+18 5.33790000e+20 2.32410000e+22 ... 3.24063005e+33
 3.29570762e+33 3.35172127e+33]


In [19]:
data_polytropic = np.column_stack((pres_out/dyn_cm_2, eps_set/g_cm_3))
np.savetxt("data_polytropic_4", data_polytropic,  fmt="%.6e")